In [ ]:
tfrecordからGPUにロードしようとするとOOM
プリフェッチが悪さしてそう

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/BELKA_valid.tfrecords.5
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/BELKA_valid.tfrecords.6
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/BELKA_valid.tfrecords.3
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/BELKA_valid.tfrecords.2
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/BELKA_valid.tfrecords.0
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/BELKA_valid.tfrecords.7
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/BELKA_valid.tfrecords.4
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/BELKA_valid.tfrecords.1
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train.tfrecords.17
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train.tfrecords.23
/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/BELKA_train.tfrecords.53
/kaggle/input/belka-train-val

In [4]:
!pip install polars

In [5]:
!pip install -q -U keras>=3

In [6]:
import os
import glob
import random
import gc
from tqdm.notebook import tqdm

import keras
from keras import layers
import jax
import jax.numpy as jnp
import numpy
import tensorflow as tf



2024-06-24 06:23:45.868598: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 06:23:45.868704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 06:23:46.000938: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
class CFG:
    DEBUG = False
    PREPROCESS = False
    EPOCHS = 15
    BATCH_SIZE = 2048
    NUM_CLASSES = 3
    NUM_TRAIN = 91_854_569
    NUM_VALID = 6_561_040
    STEPS_PER_EPOCHS_TRAIN = (NUM_TRAIN -1) // BATCH_SIZE
    STEPS_PER_EPOCHS_VALID = (NUM_VALID -1) // BATCH_SIZE

    LR = 1e-4
    WD = 0.05
    
    N_ROWS = None

    
    NBR_FOLDS = 15
    SELECTED_FOLDS = [0]
    
    ON_TPU = False
    
    if DEBUG:
        EPOCHS = 3
        BATCH_SIZE = 1024
        NBR_FOLDS = 5
        N_ROWS = 100_000
    
    SEQ_LENGTH =142
    CHANNELS = 256
    EMB_SIZE = 128
    
    
    
    SEED = 2024
    
    BASE_DIR = '/kaggle/working/'
    LOG_DIR = os.path.join(BASE_DIR, 'logs')
%mkdir {CFG.LOG_DIR}

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [8]:
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds(seed=CFG.SEED)

In [9]:
strategy = tf.distribute.MirroredStrategy()


In [10]:
trains = glob.glob('/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/train/*')
valids = glob.glob('/kaggle/input/belka-train-valid-tfrecords-1d-preprocessed/valid/*')

In [11]:
# 1件のTFRecordをデコード
def parse_example(example):
    features = tf.io.parse_single_example(
        example,
        features={
            # リストを読み込む場合は次元数を指定する
            "x": tf.io.FixedLenFeature([CFG.SEQ_LENGTH], dtype=tf.int64),
            "y": tf.io.FixedLenFeature([CFG.NUM_CLASSES], dtype=tf.float32)
        })
    x = features["x"]
    y = features["y"]
    return x, y

In [12]:
train_ds = tf.data.Dataset.from_tensor_slices(trains) \
    .interleave(
        lambda filename: tf.data.TFRecordDataset(filename).map(parse_example, num_parallel_calls=2),
        cycle_length=10) \
    .shuffle(CFG.BATCH_SIZE * 100) \
    .batch(CFG.BATCH_SIZE) \
    .prefetch(1) \
    .repeat(-1)

valid_ds = tf.data.Dataset.from_tensor_slices(valids) \
    .interleave(
        lambda filename: tf.data.TFRecordDataset(filename).map(parse_example, num_parallel_calls=2),
        cycle_length=10) \
    .shuffle(CFG.BATCH_SIZE * 100) \
    .batch(CFG.BATCH_SIZE) \
    .prefetch(1) \
    .repeat(-1)

In [13]:
next(iter(train_ds))[1].shape

TensorShape([4096, 3])

In [14]:
class ModelArgs:
    input_dim = CFG.EMB_SIZE
    channels: int = CFG.CHANNELS 
    conv_kernel_size: int = 3
    seq_length: int = 142
    num_layers: list = [8,8,2]
    dropout_rate: float = 0.2
    num_classes: int = 3
    vocab_size: int = 37
    activation: str = 'silu'
    se_reduction: int = 16 

In [15]:
class SELayer(layers.Layer):
    def __init__(self,input_size:int , reduction:int=16):
        super().__init__()
        self.avg_pool = layers.GlobalAveragePooling1D(keepdims=True)
        self.dense1 = layers.Dense(
            input_size // reduction, 
            activation='silu',
            use_bias = False
        )
        self.dense2 = layers.Dense(
            input_size, 
            activation='sigmoid',
            use_bias = False
        )
    
    def call(self, x):
        y = self.avg_pool(x)
        y = self.dense1(y)
        y = self.dense2(y)
        
        return x * y

In [16]:
class ResidualLayer(layers.Layer):
    def __init__(self, input_size:int, out_size:int, strides:int=1, pool:bool=False):
        super().__init__()
        self.hidden_size = input_size // 4
        self.norm = layers.LayerNormalization(epsilon=1e-5)
        self.conv1 = layers.Conv1D(self.hidden_size,1, strides=strides, activation='silu', padding='same', kernel_initializer='he_normal')
        self.conv2 = layers.Conv1D(self.hidden_size,3, strides=1,activation='silu', padding='same', kernel_initializer='he_normal')
        self.conv3 = layers.Conv1D(out_size, 1, strides=1, padding='same', kernel_initializer='he_normal')
        self.in_eq_out = input_size == out_size
        self.se = SELayer(out_size)
        if not self.in_eq_out :
            self.conv = layers.Conv1D(out_size,1, strides=strides, padding='same', kernel_initializer='he_normal')
        self.add = layers.Add()
            
    
    def call(self, x):
        res = x
        if not self.in_eq_out:
            res = self.conv(res)
        
        
        x = self.norm(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.se(x)
        x = self.add([x, res])
        return x        

In [17]:
def make_model(model_args:ModelArgs):    
    with strategy.scope():
        args = model_args
        inputs = keras.Input(shape=(args.seq_length,), dtype='int32')
        x = layers.Embedding(input_dim=args.vocab_size, output_dim=args.input_dim, input_length=args.seq_length, mask_zero = True)(inputs)
        x = layers.Conv1D(args.channels, 1,padding='same',activation='relu', kernel_initializer='he_normal')(x)
        x = layers.BatchNormalization()(x)

        input_cn = args.channels
        x = ResidualLayer(args.channels, args.channels*2, strides=2)(x)
        x = ResidualLayer(args.channels*2, args.channels*2)(x)
        x = ResidualLayer(args.channels*2, args.channels*2)(x)
    
        x = ResidualLayer(args.channels*2, args.channels*3, strides=2)(x)
        x = ResidualLayer(args.channels*3, args.channels*3)(x)
        x = ResidualLayer(args.channels*3, args.channels*3)(x)
    
        x = ResidualLayer(args.channels*3, args.channels*4, strides=2)(x)
        x = ResidualLayer(args.channels*4, args.channels*4)(x)
        x = ResidualLayer(args.channels*4, args.channels*4)(x)
    
#         for i, num_layer in enumerate(model_args.num_layers, 1):
#             out_cn = args.channels * i
#             x = ResidualLayer(input_cn, out_cn)(x)
#             if num_layer > 2:
#                 for j in range(num_layer-1):
#                     x = ResidualLayer(out_cn, out_cn)(x)
#             strides = 1
#             if i % 2 ==0:
#                 strides = 2
#             x = ResidualLayer(out_cn, out_cn, strides=strides)
#             input_cn = out_cn
        
        x = layers.LayerNormalization(epsilon=1e-5)(x)

        x = layers.GlobalMaxPooling1D()(x)

        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)

        outputs = layers.Dense(3, activation='sigmoid')(x)


        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay = CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model

model = make_model(ModelArgs)
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 142)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 142, 128)       │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 142, 256)       │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 142, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer (ResidualLayer)  │ (None, 71, 512)        │       226,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer_1                │ (None, 71, 512)        │       214,784 │
│ (ResidualLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer_2                │ (None, 71, 512)        │       214,784 │
│ (ResidualLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer_3                │ (None, 36, 768)        │       682,752 │
│ (ResidualLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer_4                │ (None, 36, 768)        │       481,920 │
│ (ResidualLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer_5                │ (None, 36, 768)        │       481,920 │
│ (ResidualLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer_6                │ (None, 18, 1024)       │     1,376,128 │
│ (ResidualLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer_7                │ (None, 18, 1024)       │       855,552 │
│ (ResidualLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_layer_8                │ (None, 18, 1024)       │       855,552 │
│ (ResidualLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_9           │ (None, 18, 1024)       │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 1024)           │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 7,269,763 (27.73 MB)

 Trainable params: 7,269,251 (27.73 MB)

 Non-trainable params: 512 (2.00 KB)

In [18]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=CFG.LOG_DIR)
es = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", mode='min', verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath="model-{epoch:02d}-{val_loss:.4f}.weights.h5",
                                                    save_best_only=True, save_weights_only=True,
                                                mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, verbose=1)
history = model.fit(
        train_ds,
        steps_per_epoch=CFG.STEPS_PER_EPOCHS_TRAIN,
        validation_data=valid_ds,
        validation_steps = CFG.STEPS_PER_EPOCHS_VALID,

        epochs=CFG.EPOCHS,
        callbacks=[checkpoint, reduce_lr_loss, es, tensorboard_callback],
        batch_size=CFG.BATCH_SIZE,
        verbose=1,
    )
model.load_weights(f"model-{epoch:02d}-{val_loss:.4f}.weights.h5")
oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'micro'))

preds = model.predict(test.to_numpy(), batch_size = 2*CFG.BATCH_SIZE)

Epoch 1/15


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall/functional_1_1/residual_layer_4_1/conv1d_17_1/convolution-0-0-TransposeNCHWToNHWC-LayoutOptimizer defined at (most recent call last):
<stack traces unavailable>
OOM when allocating tensor with shape[4096,1,36,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node StatefulPartitionedCall/functional_1_1/residual_layer_4_1/conv1d_17_1/convolution-0-0-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_one_step_on_iterator_27158]

In [ ]:
tst = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
tst['binds'] = 0
tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
tst[['id', 'binds']].to_csv('submission.csv', index = False)